In [4]:
import os
from py2neo import Graph, Node, Relationship
from tqdm import tqdm
from itertools import permutations



In [5]:

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")

graph_db: Graph = Graph("bolt://localhost:7687", auth=(user, password))

### lista para o algoritmos ###
* passar uma lista de pontos de interesse junto com o ponto inicial 
* calcular a distancia relativa entre os pontos
* testar todas as possibilidades de caminhos ate encontrar o menor (força bruta)

In [6]:
#achar a rua mais perto do ponto passado
def findRoadById(id: int):
    query = f'''
    MATCH  (n: Node) where id(n) = {id} return n
    '''
    return graph_db.run(query)

def findRoadId(latitude: float, longitude: float):
    query = f'''
    WITH {latitude} AS targetLat, {longitude} AS targetLon  

    MATCH (a)-[r:ROAD_TO]->(b)
    WHERE a.latitude IS NOT NULL AND a.longitude IS NOT NULL
    AND b.latitude IS NOT NULL AND b.longitude IS NOT NULL

    WITH r, a, b,
        point({{y: a.latitude, x: a.longitude}}) AS pointA,
        point({{y: b.latitude, x: b.longitude}}) AS pointB,
        point({{y: targetLat, x: targetLon}}) AS targetPoint

    WITH r, a, b,
        point.distance(pointA, targetPoint) AS distA,
        point.distance(pointB, targetPoint) AS distB

    WITH r, a, b,
        CASE WHEN distA < distB THEN distA ELSE distB END AS minDist

    ORDER BY minDist ASC
    LIMIT 1

    RETURN id(r)
    '''
    result = graph_db.run(query).data()
    if result:
        return result[0]['id(r)']
    return None

def shortPath(paramA: int, paramB: int):
    query = '''
    CALL {
        MATCH (start)
        WHERE id(start) = $paramA
        MATCH (end)
        WHERE id(end) = $paramB
        MATCH path = shortestPath((start)-[rels:ROAD_TO*]->(end))
        RETURN reduce(total = 0, r IN rels | total + r.length) AS totalLength
    }
    RETURN totalLength
    '''
    result = graph_db.run(query, paramA=paramA, paramB=paramB).data()
    if result:
        return result[0]['totalLength']
    return None

In [ ]:
# achar o ponto mais perto entre os pontos de interesse


In [16]:

initialRoad = findRoadId(latitude=-23.2848682, longitude=-47.6720885)

target_points = [
    findRoadId(latitude=-23.2999866, longitude=-47.6650897),
    findRoadId(latitude=-23.2829741, longitude=-47.6745936),
    findRoadId(-23.276087796084404, -47.67514362249426),
    findRoadId(latitude=-23.2847718, longitude=-47.6730844),
    findRoadId(latitude=-24.2847718, longitude=-47.6730844)
]

print(initialRoad, target_points)

points = [initialRoad, *target_points]

result = []
for id in points:
    subResult = []
    for subid in points:
        if id != subid:
            subResult.append({subid:shortPath(id, subid)})
    result.append(subResult)

display(result)


806 [722, 1262, 1128, 807, 147]


[[{722: 1161.179680715414},
  {1262: 4107.834821375691},
  {1128: 18026.44364496582},
  {807: 85.9036423123307},
  {147: 1755.4481158540134}],
 [{806: 1161.179680715414},
  {1262: 5267.006141372333},
  {1128: 13786.146945521474},
  {807: 1247.0833230277447},
  {147: 1133.7518922522409}],
 [{806: 4186.302544081699},
  {722: 6153.835243274004},
  {1128: 18019.600171421775},
  {807: 4272.20618639403},
  {147: 5676.962000153013}],
 [{806: 16147.846878304805},
  {722: 7805.651653053697},
  {1262: 16572.17763632855},
  {807: 16233.750520617135},
  {147: 6664.744507471153}],
 [{806: 85.9036423123307},
  {722: 1247.0833230277444},
  {1262: 4022.5637541514934},
  {1128: 17941.17257774162},
  {147: 1841.351758166344}],
 [{806: 1755.4481158540132},
  {722: 1133.7518922522409},
  {1262: 6098.686584560372},
  {1128: 13234.250080272503},
  {807: 1841.3517581663439}]]

In [19]:
# --> (n-1)!

def brute_force_shortest_path(points):
    min_distance = float('inf')
    best_path = None
    pointer = 0
    for perm in permutations(points[1:]):
        path = [points[0], *perm]
        total_distance = 0
        pointer +=1
        for i in range(len(path) - 1):
            total_distance += shortPath(path[i], path[i + 1])

        if total_distance < min_distance:
            min_distance = total_distance
            best_path = path
    print('Numero de interações', pointer)
    return best_path, min_distance


shortest_path, shortest_distance = brute_force_shortest_path(points)
print("Menor caminho:", shortest_path)
print("Menor distancia(m):", shortest_distance)

Numero de interações 120
Menor caminho: [806, 1262, 807, 722, 147, 1128]
Menor distancia(m): 23995.126303322206
